# Presto/Trino Database Connection Test

This notebook connects to the `desktop_product_intelligence_public.halley_feedback_qualtrics_comments` table using SQLAlchemy.

## Required Packages

Install required packages using UV:
```bash
uv pip install pandas sqlalchemy sqlalchemy-trino python-dotenv
```



In [2]:
## Step 2: Import Libraries and Set Up Connection

import pandas as pd
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Connection parameters
PRESTO_HOST = "presto-gdc.adp.autodesk.com"
PRESTO_PORT = 443
PRESTO_USER = os.getenv("PRESTO_USER", "your_username_here")
PRESTO_PASSWORD = os.getenv("PRESTO_PASSWORD", "your_password_here")
PRESTO_CATALOG = os.getenv("PRESTO_CATALOG", "hive")  # Set default catalog
PRESTO_SCHEMA = "desktop_product_intelligence_public"

print(f"Host: {PRESTO_HOST}")
print(f"Port: {PRESTO_PORT}")   
print(f"User: {PRESTO_USER}")
print(f"Catalog: {PRESTO_CATALOG}")
print(f"Schema: {PRESTO_SCHEMA}")


Host: presto-gdc.adp.autodesk.com
Port: 443
User: munoj
Catalog: hive
Schema: desktop_product_intelligence_public


In [3]:
## Step 3: Create Database Connection with SQLAlchemy
# This step requeries to be conneected to the company network or VPN

engine = None

try:
    connection_string = (
        f"trino://{PRESTO_USER}:{PRESTO_PASSWORD}@"
        f"{PRESTO_HOST}:{PRESTO_PORT}/"
        f"{PRESTO_CATALOG}/{PRESTO_SCHEMA}"
    )
    
    engine = create_engine(connection_string)
    
    # Test connection with a simple query
    test_query = "SELECT 1 as test"
    df_test = pd.read_sql(test_query, engine)
    
    print("✅ Connection established successfully!")
    print(f"Test query result: {df_test.values[0][0]}")
    
except Exception as e:
    print(f"❌ Connection failed: {e}")
    print(f"\nPlease check:")
    print("1. Your username and password are correct in .env file")
    print("2. You have network access to the Presto server")
    print("3. The catalog name is correct (check PRESTO_CATALOG in .env)")
    print("\nNote: You may need to install trino sqlalchemy driver:")
    print("  pip install sqlalchemy-trino")


✅ Connection established successfully!
Test query result: 1


In [4]:
## Step 4: Explore the Target Table

# Check available columns in the table

query_schema = """
DESCRIBE desktop_product_intelligence_public.halley_feedback_qualtrics_comments
"""

try:
    df_schema = pd.read_sql(query_schema, engine)
    
    print("📋 Table Columns:")
    print("-" * 80)
    for _, row in df_schema.iterrows():
        print(f"  {row[0]:<40} {row[1]}")
    print("-" * 80)
    
except Exception as e:
    print(f"❌ Error describing table: {e}")


📋 Table Columns:
--------------------------------------------------------------------------------
  delivery_channel                         varchar
  response_id                              varchar
  user_id                                  varchar
  user_language                            varchar
  product_line_name                        varchar
  insight                                  varchar
  insight_sub_type                         varchar
  insight_version                          varchar
  is_useful                                boolean
  original_comment                         varchar
  translated_comment                       varchar
  parent_topics                            array(varchar)
  topics                                   array(varchar)
  sentiment_score                          integer
  sentiment                                varchar
  product                                  varchar
  dt                                       varchar
---------------------

/var/folders/vn/5_nvqylj7yv_sf6vs4yzpzqm0000gn/T/ipykernel_33727/2640522696.py:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(f"  {row[0]:<40} {row[1]}")


In [5]:
## Step 5: Query Negative Feedback Comments

# Set the month and year for analysis
month_analysis = 9   # 1-12 for January-December
year_analysis = 2025

# Build the start date for the query (YYYY-MM-01)
start_date = f"{year_analysis}-{month_analysis:02d}-01"

print(f"📅 Analyzing data for: {start_date} (Month {month_analysis}, Year {year_analysis})")

# Dynamic query using month and year parameters
query_comments = f"""
SELECT DISTINCT
    DATE_FORMAT(parsed_date, '%Y-%m-%d') AS date_date,
    user_id,
    translated_comment,
    insight,
    insight_sub_type,
    product_line_name
FROM (
    SELECT 
        DATE_PARSE(dt, '%Y%m%d') AS parsed_date,
        user_id,
        translated_comment,
        insight,
        insight_sub_type,
        product_line_name,
        delivery_channel,
        sentiment
    FROM desktop_product_intelligence_public.halley_feedback_qualtrics_comments
    WHERE delivery_channel IN ('account_portal', 'email')
        AND sentiment IN ('Negative', 'Very Negative')
) subquery
WHERE parsed_date >= TIMESTAMP '{start_date}'
    AND parsed_date < DATE_ADD('month', 1, TIMESTAMP '{start_date}')
ORDER BY 1 DESC
LIMIT 500
"""

try:
    print("🔍 Executing query to fetch negative feedback comments...")
    df_comments = pd.read_sql(query_comments, engine)
    
    print(f"✅ Successfully fetched {len(df_comments)} rows")
    print(f"DataFrame shape: {df_comments.shape}")
    print(f"\nFirst few rows:")
    display(df_comments.head())
    
    print(f"\nData Summary:")
    print(f"Date range: {df_comments['date_date'].min()} to {df_comments['date_date'].max()}")
    print(f"Unique users: {df_comments['user_id'].nunique()}")
    print(f"\nInsight distribution:")
    print(df_comments['insight'].value_counts())

except Exception as e:
    print(f"❌ Error executing query: {e}")
    print(f"\nPlease check:")
    print("1. The connection is still active")
    print("2. The table and columns exist")
    print("3. The date range contains data")




📅 Analyzing data for: 2025-09-01 (Month 9, Year 2025)
🔍 Executing query to fetch negative feedback comments...
✅ Successfully fetched 59 rows
DataFrame shape: (59, 6)

First few rows:


,date_date,user_id,translated_comment,insight,insight_sub_type,product_line_name
0,2025-09-27,BD323G88QM849JF3,We tried it and it doesn't work,FEATURE_RECOMMENDATION,WALLS_AUTO_JOIN_LOCK,RVT
1,2025-09-27,BD323G88QM849JF3,The excavation tool is not at all reliable unf...,OPTIMIZER_RECOMMENDATION,TOPOSOLID_VOID_CUT_STABILITY,RVT
2,2025-09-27,BD323G88QM849JF3,This data is not really representing user expe...,TEAM_PERFORMANCE_INFO,DEFAULT,ACD
3,2025-09-26,4QVGRZGRJCUR,useless,SKILLTREE,SKILLTREE,ACD
4,2025-09-26,QMK2TDQCQLBY2QYL,Not at all useful.,COMMAND_RECOMMENDATION,ID_OBJECTS_CLOUD,RVT



Data Summary:
Date range: 2025-09-01 to 2025-09-27
Unique users: 47

Insight distribution:
insight
FEATURE_RECOMMENDATION      11
TEAM_PERFORMANCE_INFO       11
COMMAND_RECOMMENDATION      10
USAGE_BY_CATEGORY            7
SKILLTREE                    6
REPEATABLE_COMMANDS          5
HAVE_YOU_TRIED               3
PERFORMANCE_INFO             2
OPTIMIZER_RECOMMENDATION     1
WHATS_NEW                    1
BEST_PRACTICES               1
FILE_REPORT                  1
Name: count, dtype: int64


In [5]:
## Step 6: Close Connection

# Clean up the database connection when done

try:
    engine.dispose()
    print("✅ Connection closed successfully")
except:
    print("⚠️ Connection was already closed or not established")


✅ Connection closed successfully
